In [13]:
import base64
import logging
import re
import uuid
from pprint import pprint

import requests

import matplotlib.pyplot as plt
import numpy as np
import urllib

In [14]:
USERNAME= "Tytti.vilen@metropolia.fi"
PASSWORD= "Tua123456789"
CLIENT_ID= "74571pdhuc7vvak4tl45uts8u8"

In [15]:
LOGIN_URL = "https://kubioscloud.auth.eu-west-1.amazoncognito.com/login"
TOKEN_URL = "https://kubioscloud.auth.eu-west1.amazoncognito.com/oauth2/token"
REDIRECT_URI = "https://analysis.kubioscloud.com/v1/portal/login"
USER_AGENT = "TTT"

In [16]:
# Logging info
logging.basicConfig(format="%(asctime)-15s [%(levelname)s]: %(message)s")

log = logging.getLogger(__name__)
log.setLevel(logging.INFO)

csrf = str(uuid.uuid4())

# Login data structure
login_data = {
    "client_id": CLIENT_ID,
    "redirect_uri": REDIRECT_URI,
    "username": USERNAME,
    "password": PASSWORD,
    "response_type": "code",
    "access_type": "offline",
    "_csrf": csrf,
}

print(login_data)
# Start a session
session = requests.session()

# Open a session
log.info("Authenticating to '%r' with client_id: %r", LOGIN_URL, CLIENT_ID)
login_response = session.post(
    LOGIN_URL,
    data=login_data,
    allow_redirects=False,
    headers={"Cookie": f"XSRF-TOKEN={csrf}", "User-Agent": USER_AGENT},
) 
log.info(login_response)
assert (
    login_response.status_code == 302
), f"Status: {login_response.status_code}, Authentication failed."
log.info(login_response.headers["Location"])
code = login_response.headers["Location"].split("=")[1]
log.info("Got code: %r", code)

# Exchange tokens
log.info("Exchanging code to tokens")
exch_data = {
    "client_id": CLIENT_ID,
    "code": code,
    "redirect_uri": REDIRECT_URI,
    "grant_type": "authorization_code",
}
exch_response = session.post(
    TOKEN_URL, data=exch_data
)
log.info("Status code %r", exch_response.status_code)
tokens = exch_response.json()

# What are the possible keys?
log.info(tokens.keys())

# When do these tokens expire?
log.info(f"Tokens expires in: {tokens['expires_in']} seconds.")


2024-03-08 13:58:02,180 [INFO]: Authenticating to ''https://kubioscloud.auth.eu-west-1.amazoncognito.com/login'' with client_id: '74571pdhuc7vvak4tl45uts8u8'


{'client_id': '74571pdhuc7vvak4tl45uts8u8', 'redirect_uri': 'https://analysis.kubioscloud.com/v1/portal/login', 'username': 'Tytti.vilen@metropolia.fi', 'password': 'Tua123456789', 'response_type': 'code', 'access_type': 'offline', '_csrf': 'd0040eef-7b23-4f94-ae0a-ad60b18b8e2b'}


2024-03-08 13:58:03,041 [INFO]: <Response [302]>
2024-03-08 13:58:03,046 [INFO]: https://analysis.kubioscloud.com/v1/portal/login?code=f369b2bb-7043-44f9-967a-84308cd0f11b
2024-03-08 13:58:03,048 [INFO]: Got code: 'f369b2bb-7043-44f9-967a-84308cd0f11b'
2024-03-08 13:58:03,051 [INFO]: Exchanging code to tokens


ConnectionError: HTTPSConnectionPool(host='kubioscloud.auth.eu-west1.amazoncognito.com', port=443): Max retries exceeded with url: /oauth2/token (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000020639D23A90>: Failed to resolve 'kubioscloud.auth.eu-west1.amazoncognito.com' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
HEADERS = {"Authorization": tokens["id_token"], "User-Agent": USER_AGENT}

BASE_URL = "https://analysis.kubioscloud.com"

GET_USER_INFO = BASE_URL + "/v2/user/self"
GET_RESULT = BASE_URL + "/v2/result/self"
GET_DAILY_READINESS = BASE_URL + "/v2/result/self?types=readiness&daily=yes"

## Return personal information for the currently authenticated user ##
log.info("Get user info")
response = session.get(GET_USER_INFO, headers = HEADERS)
user_info = response.json()

## To list all results for the current user from the last 30 days (all defaults) ##
log.info("Get results")
response = session.get(GET_RESULT, headers = HEADERS)
result = response.json()

## To list only “daily” readiness results ##
log.info("Get daily readiness results")
response = session.get(GET_DAILY_READINESS, headers = HEADERS)
daily_readiness = response.json()

In [ ]:
readiness = []
created = []
stress_index = []

print('  n  Ready Stress   Timestamp')

for n, r in enumerate(result['results']):
    r_data = r['result']['readiness']
    c_stamp = r['create_timestamp']
    stress_i = r['result']['stress_index']
    readiness.append(r_data)
    created.append(c_stamp)
    stress_index.append(stress_i)
    print(f'{n:3d}  {r_data:5.0f}  {stress_i:5.1f}   {c_stamp}')


In [ ]:
## Read the DATA_URL and 'measured_timestamp'
DATA_URL = details['measure']['channels'][0]['data_url']
measured = details['measure']['measured_timestamp']

## Read the RR/PPI data from DATA_URL
data = urllib.request.urlopen(DATA_URL)
byte = data.read(2)
rr = []

## Convert the binary data to numpy array
while byte:
    rr.append(int.from_bytes(byte, byteorder = "little"))
    byte = data.read(2)
rr = np.array(rr)

In [ ]:
## Time is the cumulative sum of rr-data
x = np.cumsum(rr)/1000

plt.plot(x, rr)
plt.xlabel('time (s)')
plt.ylabel('RR (ms)')
plt.title(measured)
plt.grid()
plt.show()

In [ ]:
## Heart rate is the inverse of rr (ms)
bpm = 60*1000/rr

plt.plot(x, bpm)
plt.xlabel('time (s)')
plt.ylabel('Heart rate (BPM)')
plt.title(measured)
plt.grid()
plt.show()

In [ ]:
## Heart rate is the inverse of rr (ms)
bpm = 60*1000/rr

plt.plot(x, bpm)
plt.xlabel('time (s)')
plt.ylabel('Heart rate (BPM)')
plt.title(measured)
plt.grid()
plt.show()